# CTC version 1




### Paper Reference
- [Connectionist Temporal Classification (IDSIA)](ftp://ftp.idsia.ch/pub/juergen/icml2006.pdf)


### English Test
First we test English



In [1]:
# Include And 
import sys
sys.path.append("/home/pika/nntools/")

import theano
import theano.tensor as T
import numpy as np
from theano_toolkit import utils as U
from theano_toolkit import updates
from theano.printing import Print
from theano_toolkit.parameters import Parameters

import lasagne
from lasagne.layers import *

from time import time

import ctc_cost_2

import cPickle
import sys
sys.setrecursionlimit(100000)

from random import shuffle

Using gpu device 0: GeForce GTX 980
/home/pika/nntools/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


# Data


In [2]:
'''
Doing : 
    - Read data and return x_train, y_train
'''
# TRAINING_ACOUSTIC_FEATURE = "./dataCebuano/train.f0_ffv_fbank.fea"
# TRAINING_ACOUSTIC_FEATURE = "./dataCebuano/train_Normalized_SELFVALID.ark"
TRAINING_ACOUSTIC_FEATURE = "./dataCebuano/tlkagk/train.dnn.fea.lessThen500.ark"
TRAINING_LABELS = "./dataCebuano/train.txt"
CHAR_CORPUS = "./dataCebuano/corpus.txt"


def readDataSet () :
    
    # -- Mapping char to integer and reverse
    char_map = {}    # eg. { a -> 1 }
    char_unMap = {}  # eg. { 1 -> a }
    with open( CHAR_CORPUS, 'r' ) as f:
        char_index = 0
        for lines in f:
            char_map  [ lines[:-1] ] = char_index
            char_unMap[ char_index ] = lines[:-1]
            char_index += 1
        # add Blank, take '.' as blank (not space)
        char_map['.'] = char_index
        char_unMap[char_index] = '.'
    
    
    # -- Reading X = id -> [ list of feature ] 
    idMapX = {}
    # idDeck = []
    id_valid = []
    id_train = []
    id = ''
    
    tmp_l = 0;
    with open( TRAINING_ACOUSTIC_FEATURE, 'r') as f:
        for lines in f: 
            if '[' in lines :
                id = lines.split(' ')[0]
                if id.split('_')[0] == "10647" or id.split('_')[0] == "11581" or id.split('_')[0] == "11673":
                    id_valid.append(id)
                else :
                    id_train.append(id)
                idMapX[id] = []
            elif ']' in lines :
                if( len( lines.split('\n')[0].split(' ') [0:-1] ) != 90 ) :
                    print "err"
                    return 
                idMapX[id].append([float(x) for x in lines.split('\n')[0].split(' ') [:-1] ])
            else :
                if( len( lines.split('\n')[0].split(' ')) != 90 ):
                    print len( lines.split('\n')[0].split(' '))
                    print lines.split('\n')[0].split(' ')
                    print lines
                    print id
                    print "fuck"
                    return
                idMapX[id].append([float(x) for x in lines.split('\n')[0].split(' ') ])
    
    # Move to  3 - main - 3
    '''
    monoDimension = 33;
    zeroFeature = [0] * monoDimension
    #print len( zeroFeature ) 
    #print zeroFeature
    for id in idDeck:
        # print id
        tmpIdMapXWithID = []
        if len(idMapX[id]) < 6 :
            print "ERROR in length"
            return 
        for idxX in xrange( len(idMapX[id]) ):
            if idxX == 0:
                tmpIdMapXWithID.append( zeroFeature + zeroFeature + zeroFeature + zeroFeature + zeroFeature + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + idMapX[id][idxX+5] )    
            elif idxX == 1:
                tmpIdMapXWithID.append( zeroFeature + zeroFeature + zeroFeature + zeroFeature + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + idMapX[id][idxX+5] )  
            elif idxX == 2:
                tmpIdMapXWithID.append( zeroFeature + zeroFeature + zeroFeature + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + idMapX[id][idxX+5] )  
            elif idxX == 3:
                tmpIdMapXWithID.append( zeroFeature + zeroFeature + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + idMapX[id][idxX+5] )  
            elif idxX == 4:
                tmpIdMapXWithID.append( zeroFeature + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + idMapX[id][idxX+5] )  
            
            elif idxX == (len(idMapX[id]) - 5 ):
                tmpIdMapXWithID.append( idMapX[id][idxX-5] + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + zeroFeature )  
            elif idxX == (len(idMapX[id]) - 4 ):
                tmpIdMapXWithID.append( idMapX[id][idxX-5] + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + zeroFeature + zeroFeature )  
            elif idxX == (len(idMapX[id]) - 3 ):
                tmpIdMapXWithID.append( idMapX[id][idxX-5] + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + zeroFeature + zeroFeature + zeroFeature )  
            elif idxX == (len(idMapX[id]) - 2 ):
                tmpIdMapXWithID.append( idMapX[id][idxX-5] + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + zeroFeature + zeroFeature + zeroFeature + zeroFeature )  
            elif idxX == (len(idMapX[id]) - 1 ):
                tmpIdMapXWithID.append( idMapX[id][idxX-5] + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + zeroFeature + zeroFeature + zeroFeature + zeroFeature + zeroFeature )  
            else:
                tmpIdMapXWithID.append( idMapX[id][idxX-5] + idMapX[id][idxX-4] + idMapX[id][idxX-3] + idMapX[id][idxX-2] + idMapX[id][idxX-1] + idMapX[id][idxX] + idMapX[id][idxX+1] + idMapX[id][idxX+2] + idMapX[id][idxX+3] + idMapX[id][idxX+4] + idMapX[id][idxX+5] )  
        idMapX[id] = tmpIdMapXWithID
    '''
    
    print "Concate"
    
    # Standardlize X (Done in PrepreProcessing )
   
    # Shuffle the speaker to slice training and validation
    '''
    totalSpeech = len(idMapX)
    slicePointFive = totalSpeech // 20
    shuffle(idDeck)
    id_train = idDeck[slicePointFive+1:]
    id_valid = idDeck[:slicePointFive]
    '''
    idMapY = {} 
    
    # -- Reading Y = id -> [ Sentence(list of char[int] ) ]
    with open( TRAINING_LABELS, 'r') as f:
        specific = '?'
        for lines in f:
            yWithId = lines.split('\n')[0].split(' ', 1)
            idMapY[ yWithId[0] ] = []
            judgeSpecific = False
            for char in yWithId[1] :
                if char == '<' :
                    judgeSpecific = True;                
                if( judgeSpecific ):
                    if char == '>':
                        idMapY[ yWithId[0] ].append( char_map[specific] )
                        judgeSpecific = False
                else :
                    idMapY[ yWithId[0] ].append( char_map[char] )
    
    # -- concatenate X and Y
    x_train = []
    x_valid = []
    y_train = []
    y_valid = []
    
    for id in id_train:
        concateId = []
        for i in range( len( idMapX[id] ) ) :
            concateId.append( idMapX[id][i] )
        x_train.append( floatX( concateId )  )
        y_train.append( floatX( idMapY[id]) )
        
    for id in id_valid:
        concateId = []
        for i in range( len( idMapX[id] ) ) :
            concateId.append( idMapX[id][i] )
        x_valid.append( floatX( concateId ) )
        y_valid.append( floatX( idMapY[id]) )
        
    print "Dataset Finish : "
    print "total x_train = " + str(len(x_train))
    print x_train[0]
    print "total y_train = " + str(len(y_train))
    print y_train[0]
    print "total x_valid = " + str(len(x_valid))
    print x_valid[0]
    print "total y_valid = " + str(len(y_valid))
    print y_valid[0]
    
    return char_map, char_unMap, x_train, y_train, x_valid, y_valid

def floatX(x):
    return np.asarray(x, dtype=theano.config.floatX)


char_map, char_unMap, x_train, y_train, x_valid, y_valid = readDataSet()

Concate
Dataset Finish : 
total x_train = 3851
[[ 0.30875856  3.77019548  0.02795531 ...,  0.12686451  0.15877695
  -0.94809389]
 [ 0.54844403  2.00368309  1.21554422 ...,  0.08752131 -0.99940634
   0.11953995]
 [ 0.74679321  1.01089871  1.99684393 ..., -0.29566044 -2.02656507
   0.68761522]
 ..., 
 [ 1.76844943 -1.40381742 -0.66371351 ..., -0.63197482 -0.1331615
   0.24962384]
 [ 1.93053758 -1.46547914 -0.39516619 ..., -0.40233928  0.06236838
  -0.19390136]
 [ 2.00433135 -1.49505806 -0.54028153 ..., -0.18457986  0.12119711
  -0.42793319]]
total y_train = 3851
[ 24.  19.  22.   5.   0.   9.  12.   5.  28.   0.  18.   2.   0.  24.  19.
  22.   5.   0.   9.  12.   5.  28.   0.   9.   5.  28.   0.  18.  11.  11.
   0.  19.  12.   0.  24.  19.   6.  17.   8.   0.  16.  12.  18.   0.   9.
  12.   5.  28.]
total x_valid = 67
[[ 0.91447097  4.14750957 -0.04186375 ...,  0.25524774  0.18128966
   0.78523731]
 [ 0.94787949  3.73508501 -0.03330386 ...,  0.12123027  0.25109428
   0.96071202]
 [ 1.

In [3]:
# 500 -> 3722 / 195
# 600 -> 3821 / 201
# 3000 -> 3937 / 207
x_train[0].shape

(302, 90)

In [4]:
max( [X.shape[0] for X in x_train] )

501

In [5]:
'''
    https://github.com/craffel/lstm_benchmarks/blob/master/lasagne/experiment.py
    
    Convert a list of matrices into batches of uniform length
    :parameters:
        - X : list of np.ndarray
            List of matrices
        - length : int
            Desired sequence length.  Smaller sequences will be padded with 0s,
            longer will be truncated.
        - batch_size : int
            Mini-batch size
    :returns:
        - X_batch : np.ndarray
            Tensor of time series matrix batches,
            shape=(n_batches, length, batch_size, n_features)
        - X_mask : np.ndarray
            shape=(n_batches, length, batch_size)
            Mask denoting whether to include each time step of each time series
            matrix
    '''
    
def make_batches_X(X, length, batch_size=30):
    n_batches = len(X)//batch_size
    X_batch = np.zeros( (n_batches, batch_size, length, X[0].shape[1]),
                         dtype=theano.config.floatX)
    X_mask  = np.zeros( (n_batches, length, batch_size ), 
                         dtype=theano.config.floatX)
    
    for b in range(n_batches): 
        for n in range(batch_size): # go thorough batch size       
            X_m = X[b*batch_size + n] # seq_length X feature dim            
            X_batch[b, n, :X_m.shape[0]] = X_m[:length]
            X_mask[b, :X_m.shape[0], n] = 1
            
    return X_batch, X_mask

def make_batches_Y( X, length, batch_size=30):
    n_batches = len(X)//batch_size
    
    X_batch = np.zeros( (n_batches, length, batch_size ), dtype='float32')
    
    X_mask = np.zeros(X_batch.shape, dtype=theano.config.floatX)
    
    for b in range(n_batches):
        for n in range(batch_size):
            X_m = X[ b*batch_size + n ]
            X_batch[b, :X_m.shape[0], n ] = X_m[:length]
            X_mask[b, :X_m.shape[0], n] = 1
    return X_batch, X_mask




In [6]:
batch_size = 40

# Find the longest sequence
length_x = max( max( [X.shape[0] for X in x_train] ),
                max( [X.shape[0] for X in x_valid] ))

length_y = max( max( [X.shape[0] for X in y_train] ),
                max( [X.shape[0] for X in y_valid] ))

# Convert to batches of time series of uniform length
# x_train_mask: seq_length X batch_size
# y_train_mask: output_length X batch_size
# y_pred_mask = x_train_mask, since pred by sequence
x_train, x_train_mask = make_batches_X(x_train, length_x, batch_size)
y_train, y_train_mask = make_batches_Y(y_train, length_y, batch_size)
x_valid, x_valid_mask = make_batches_X(x_valid, length_x, batch_size)
y_valid, y_valid_mask = make_batches_Y(y_valid, length_y, batch_size)



In [7]:
print x_train.shape
print y_train.shape
print x_valid.shape
print y_valid.shape
print "================mask================"
print x_train_mask.shape
print y_train_mask.shape

# print x_train_mask[0][400][0]

(96, 40, 501, 90)
(96, 132, 40)
(1, 40, 501, 90)
(1, 132, 40)
================mask================
(96, 501, 40)
(96, 132, 40)


# Model Part


In [8]:
'''
credit by NLTK package of measure edit distance
'''

def _edit_dist_init(len1, len2):
    lev = []
    for i in range(len1):
        lev.append([0] * len2)  # initialize 2D array to zero
    for i in range(len1):
        lev[i][0] = i           # column 0: 0,1,2,3,4,...
    for j in range(len2):
        lev[0][j] = j           # row 0: 0,1,2,3,4,...
    return lev


def _edit_dist_step(lev, i, j, s1, s2, transpositions=False):
    c1 = s1[i - 1]
    c2 = s2[j - 1]

    # skipping a character in s1
    a = lev[i - 1][j] + 1
    # skipping a character in s2
    b = lev[i][j - 1] + 1
    # substitution
    c = lev[i - 1][j - 1] + (c1 != c2)

    # transposition
    d = c + 1  # never picked by default
    if transpositions and i > 1 and j > 1:
        if s1[i - 2] == c2 and s2[j - 2] == c1:
            d = lev[i - 2][j - 2] + 1

    # pick the cheapest
    lev[i][j] = min(a, b, c, d)


def check_label_error( real , predict, transpositions=False):
    ## length of real >= length of predict
    """
    Calculate the Levenshtein edit-distance between two strings.
    The edit distance is the number of characters that need to be
    substituted, inserted, or deleted, to transform s1 into s2.  For
    example, transforming "rain" to "shine" requires three steps,
    consisting of two substitutions and one insertion:
    "rain" -> "sain" -> "shin" -> "shine".  These operations could have
    been done in other orders, but at least three steps are needed.

    This also optionally allows transposition edits (e.g., "ab" -> "ba"),
    though this is disabled by default.

    :param s1, s2: The strings to be analysed
    :param transpositions: Whether to allow transposition edits
    :type s1: str
    :type s2: str
    :type transpositions: bool
    :rtype int
    """
    # set up a 2-D array
    len1 = len(predict)
    len2 = len(real)
    lev = _edit_dist_init(len1 + 1, len2 + 1)

    # iterate over the array
    for i in range(len1):
        for j in range(len2):
            _edit_dist_step(lev, i + 1, j + 1, predict, real, transpositions=transpositions)
            
    return lev[len1][len2]*1.0/len2


def clean_up( y ):
    """
    for final output clean up
    B(a − ab−) = B(−aa − −abb) = aab
    """
    answer = ""

    for i in range(len(y)):
        if y[i] == '.':
            continue
        else:
            if y[i-1] != y[i]:
                answer += y[i]
    
    return answer

def remap_back( y , actual):
    answer = ""
    
    for i in y:
        answer += char_unMap[i]
    
    if not actual:
        answer = clean_up(answer)
        
    return answer

def decode_all_actual( y, y_mask ,batch_size , actual=False):
    """
    y     : label_length X batch_size
    y_mask: label_length X batch_size
    """
    result = []
    for i in xrange(batch_size):
        mask = np.swapaxes( y_mask, 0, 1)[i]
        ans = np.swapaxes( y, 0 , 1)[i]
        result.append(remap_back( ans[np.nonzero(mask)] , actual ))
    return result


def decode_all_pred( y, y_mask ,batch_size , actual=False):
    """
    y     : label_length X batch_size
    y_mask: label_length X batch_size
    """
    result = []
    for i in xrange(batch_size):
        mask = np.swapaxes( y_mask, 0, 1)[i]
        ans = y[i]
        result.append(remap_back( ans[np.nonzero(mask)] , actual ))
    return result

def check_all( y, y_mask, y_pred, y_pred_mask, batch_size ):
    
    actual = decode_all_actual( y, y_mask, batch_size , True )
    predict = decode_all_pred( y_pred, y_pred_mask, batch_size , False )
    
    error = 0.
    for a,b in zip (actual, predict):
        error += check_label_error(a,b)
    
    return error/len(actual)

In [9]:
class end_to_end():  
    def __init__(self, input_shape, max_seq_length, hidden_layer,
                 batch, max_epochs, output_num_units,
                 patience, up_learning_rate, file_name):
        
        self.input_shape = input_shape # [batch, dim]
        
        self.hidden_layer = hidden_layer # hidden [l1, l2, l3]
        self.output_num_units = output_num_units # [ # of class ]
        
        self.batch = batch
        self.max_epochs = max_epochs
        
        self.up_learning_rate = up_learning_rate
                 
        self.patience = patience
        self.best_valid = np.inf
        self.best_valid_epoch = 0
        self.best_params = None
        
        self.train_history_ = []
        self.epochs = 0
        
        self.file_name = file_name
        
        self.max_seq_length = max_seq_length
        
        """
        input data type
        y_hat : T x B x C+1
        y : L x B
        y_hat_mask : T x B
        y_mask : L x B
        """
        
        print "Why model build so long ... "
        # T x B x F
        # B X T X F (Lasagne format)
        x = T.tensor3('X', dtype=theano.config.floatX)
        # L x B
        y = T.matrix ('y', dtype=theano.config.floatX)

        # L x B
        y_mask = T.matrix('y_mask', dtype=theano.config.floatX)
        # T x B
        y_hat_mask = T.matrix('y_hat_mask', dtype=theano.config.floatX)
    
        # Min/max sequence length
        MAX_LENGTH = max_seq_length
        
        # Number of training sequences in each batch
        N_BATCH = batch
        
        #===========================================================================================
        # dEEP lSTM
        #===========================================================================================
        print "Input : (N_BATCH) : " + str(N_BATCH) +", ML :" + str(MAX_LENGTH) + "ss :" + str(self.input_shape[1])
        
        # Recurrent layers expect input of shape
        # (batch size, max sequence length, number of features)
        l_in     = lasagne.layers.InputLayer( shape=( N_BATCH, MAX_LENGTH, self.input_shape[1] ) )
        l_in_gau = lasagne.layers.GaussianNoiseLayer( l_in, sigma=0.5 )
        
        # LSTM layer 1
        l_forward_1   = lasagne.layers.LSTMLayer(l_in_gau, num_units=hidden_layer[0], learn_init=True, peepholes=True)
        l_backward_1  = lasagne.layers.LSTMLayer(l_in_gau, num_units=hidden_layer[0], backwards=True, learn_init=True, peepholes=True)
        l_recurrent_1 = ElemwiseSumLayer( [l_forward_1, l_backward_1] )

        
        # LSTM layer 2
        l_forward_2   = lasagne.layers.LSTMLayer(l_recurrent_1, num_units=hidden_layer[1], learn_init=True, peepholes=True)
        l_backward_2  = lasagne.layers.LSTMLayer(l_recurrent_1, num_units=hidden_layer[1], backwards=True, learn_init=True, peepholes=True)
        l_recurrent_2 = ElemwiseSumLayer( [l_forward_2, l_backward_2] )
        
        '''
        # LSTM layer 3
        l_forward_3   = lasagne.layers.LSTMLayer(l_recurrent_2, num_units=hidden_layer[2], learn_init=True, peepholes=True)
        l_backward_3  = lasagne.layers.LSTMLayer(l_recurrent_2, num_units=hidden_layer[2], backwards=True, learn_init=True, peepholes=True)
        l_recurrent_3 = ElemwiseSumLayer( [l_forward_3, l_backward_3] )
        
        # LSTM layer 4
        l_forward_4   = lasagne.layers.LSTMLayer(l_recurrent_3, num_units=hidden_layer[3], learn_init=True, peepholes=True)
        l_backward_4  = lasagne.layers.LSTMLayer(l_recurrent_3, num_units=hidden_layer[3], backwards=True, learn_init=True, peepholes=True)
        l_recurrent_4 = ElemwiseSumLayer( [l_forward_4, l_backward_4] )
        '''
        
        # l_reshape = lasagne.layers.ReshapeLayer(l_recurrent_4, (N_BATCH*MAX_LENGTH, hidden_layer[3])  )
        l_reshape = lasagne.layers.ReshapeLayer(l_recurrent_2, (N_BATCH*MAX_LENGTH, hidden_layer[1])  )

        
        #===========================================================================================
        # COMMON SETUP
        #===========================================================================================
        
        # Our output layer is a simple dense connection
        l_recurrent_out      = lasagne.layers.DenseLayer( l_reshape, num_units=output_num_units[0] , nonlinearity=lasagne.nonlinearities.identity)
        
        # Now, reshape the output back to the RNN format
        l_out_shp            = lasagne.layers.ReshapeLayer( l_recurrent_out, (N_BATCH, MAX_LENGTH, output_num_units[0]) )
        
        # dimshuffle to shape format (input_seq_len, batch_size, num_classes + 1)
        l_out_shp_ctc        = lasagne.layers.DimshuffleLayer( l_out_shp, (1, 0, 2))

        l_out_softmax        = lasagne.layers.NonlinearityLayer( l_recurrent_out, nonlinearity=lasagne.nonlinearities.softmax)
        l_out_softmax_shp    = lasagne.layers.ReshapeLayer( l_out_softmax, (N_BATCH, MAX_LENGTH, output_num_units[0] ))
        
        # since we use gaussian noise in input, False means use noise, True means dont use noise
        output_lin_ctc_train = lasagne.layers.get_output( l_out_shp_ctc, x, deterministic=False)
        output_softmax_train = lasagne.layers.get_output( l_out_softmax_shp, x, deterministic=False)
        
        output_lin_ctc_val   = lasagne.layers.get_output(l_out_shp_ctc, x, deterministic=True)
        output_softmax_val   = lasagne.layers.get_output(l_out_softmax_shp, x, deterministic=True)
        
        self.all_params      = lasagne.layers.get_all_params(l_out_shp)
    
        # the CTC cross entropy between y and linear output network
        pseudo_cost = ctc_cost_2.pseudo_cost(
            y, output_lin_ctc_train, y_mask, y_hat_mask,
            skip_softmax=True)
        
        
        pseudo_cost_grad = T.grad(pseudo_cost.mean(), self.all_params)
        true_cost        = ctc_cost_2.cost(y, output_softmax_train.dimshuffle(1, 0, 2), y_mask, y_hat_mask)
        cost             = T.mean(true_cost)
        
        pseudo_cost_scaled_grads = lasagne.updates.total_norm_constraint(pseudo_cost_grad, 5)
        updates_rms          = lasagne.updates.rmsprop(pseudo_cost_scaled_grads, self.all_params, learning_rate = self.up_learning_rate)
        updates = lasagne.updates.apply_momentum(updates_rms,  self.all_params, momentum=0.9)
        
        self.train = theano.function(
            inputs = [x, y, y_hat_mask, y_mask],
            outputs = [ pseudo_cost.mean(), cost, output_softmax_train ],
#             outputs = [output_lin_ctc, output_softmax, cost],
            updates=updates
        )
        
        self.predict = theano.function( 
            inputs=[x], 
            outputs = [ output_softmax_val] 
        )

        
    # x_mask and y_mask is the same
    # x_test_mask and y_test_mask is the same
    def fit(self, x_train, y_train, x_test,  y_test , x_mask, y_mask, x_test_mask, y_test_mask ):
        print " "
        print "start training!!!!"
        print " "
        sys.stdout.flush()
        
        for i in range(self.max_epochs):
            self.epochs +=1
            t0 = time()
            
            cs = 0.
            pseudo_cs = 0.
            for index in range(len(x_train)):
                pseudo, cost, output_softmax = self.train( x_train[index] , y_train[index],
                                                          x_mask[index], y_mask[index])
                cs += cost
                pseudo_cs += pseudo
                gg = index
                if index % 10 == 0:
                    print index, pseudo, cost
                sys.stdout.flush()
            
            cs /= len(x_train)
            pseudo_cs /= len(x_train)
            
#             cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
            if self.epochs <= 1:
                previous_cs = "-"
            else:
                previous_cs = self.train_history_[-1]['cost']
            print "\n==============================="
            print 'epoch {0} : pseudo= {1}, cost= {2}, previous_cost= {3}, train_time = {4} s'.format(self.epochs,
                                                                                                      pseudo_cs, 
                                                                                                      cs,
                                                                                                      previous_cs, 
                                                                                                      time() - t0)
            
            sys.stdout.flush()
            
            t0 = time()
            
#             print np.argmax(output_softmax[:],axis=2)
#             print decode_all( np.argmax(output_softmax[:],axis=2) , x_train_mask[gg],  batch_size , True)
            
            # save model first
            # dont know the LER performance yet
#             cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
    

            label_error_rate = 0.
            print len(x_test)
            y_actual = []
            y_predict = []
            for index in range( len(x_test) ) :

                prepre = self.predict( x_test[index])
                
                label_error_rate += check_all( y_test[index], y_test_mask[index],
                                              np.argmax(prepre[0],axis=2) , x_test_mask[index],
                                              batch_size)
                
                if (( index + self.epochs ) % 4 == 0 ):
                    ## print actual
                    y_actual = decode_all_actual( y_test[index], y_test_mask[index], batch_size , True)

                    ## print pred
                    y_predict = decode_all_pred( np.argmax(prepre[0],axis=2) , x_test_mask[index],  batch_size , False)
                
            label_error_rate /= len(x_test)
            self.train_history_.append({"epoch":self.epochs, "cost": cs, "LER":label_error_rate})
            
            print '\t\t\t val_= {0}, test_time  = {1} s'.format(label_error_rate, time() - t0)
            print "===============================\n"
            
            sys.stdout.flush()
            
            for a , b in zip (y_actual,y_predict):
                print "Target== ",a 
                print "\tAns => ",b
            
            sys.stdout.flush()
            
            """
            should use cost do early stopping
            """
            current_cs = self.train_history_[-1]['cost']
            current_epoch = self.train_history_[-1]['epoch']
            if current_cs < self.best_valid:
                self.best_valid = current_cs
                self.best_valid_epoch = current_epoch
                self.best_params = [w.get_value() for w in self.all_params]
                cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
                
            elif self.best_valid_epoch + self.patience <= current_epoch:
                print ""
                print "Early stopping."
                print self.best_valid_epoch,self.best_valid
                print "Best valid ler {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)              
#                 for qq in range (len(self.all_params)):
#                     self.all_params[qq].set_value( self.best_params[qq] )
#                 break

            """
            can not use label error rate do early stopping
            """
#             current_ler = self.train_history_[-1]['LER']
#             current_epoch = self.train_history_[-1]['epoch']
#             if current_ler < self.best_valid:
# #                 print "********************* Now best ************************"
#                 sys.stdout.flush()
#                 self.best_valid = current_ler
#                 self.best_valid_epoch = current_epoch
#                 self.best_params = [w.get_value() for w in self.all_params]
# #                 cPickle.dump( self , open("./"+self.file_name+".pkl","wb"))
                
#             elif self.best_valid_epoch + self.patience <= current_epoch:
#                 print ""
#                 print "Early stopping."
#                 print self.best_valid_epoch,self.best_valid
#                 print "Best valid ler {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)
#                 sys.stdout.flush()                
#                 for qq in range (len(self.all_params)):
#                     self.all_params[qq].set_value( self.best_params[qq] )
#                 break


#     def prediction(self, x, x_mask) :
        
#         abc =  self.predict(x, x_mask)
        
#         return np.argmax(abc[0], axis = 1 )



In [10]:
'''
ONE lAYER
'''

model2L_BRAND_Noise_momemtumClip_bottleneckfea = end_to_end (
    input_shape      = (1, x_train.shape[3] ) , # batch of 1, (110, 30, 777, 117)
    max_seq_length = x_train.shape[2],
    hidden_layer     = [ 650, 650, 650, 650 ], # maximum layer to LSTM 3 layer only
    batch            = batch_size, 
    max_epochs       = 300, 
    output_num_units = [ len(char_unMap) ],
    up_learning_rate = 0.0001, 
    patience         = 7,
    file_name = "model2L_BRAND_Noise_momemtumClip_bottleneckfea"
)


/home/pika/nntools/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "
/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan.py:1017: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


Why model build so long ... 
Input : (N_BATCH) : 40, ML :501ss :90


/home/pika/anaconda/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [11]:
%time model2L_BRAND_Noise_momemtumClip_bottleneckfea.fit( x_train, y_train, x_valid, y_valid, x_train_mask, y_train_mask , x_valid_mask, y_valid_mask )

 
start training!!!!
 
0 -41.3696784973 701.955688477
10 184.33454895 239.494064331
20 1061.46081543 inf
30 1076.11035156 inf
40 838.116638184 940.44519043
50 1061.4074707 1301.92089844
60 1320.21228027 inf
70 895.698852539 926.50958252
80 1009.78448486 1086.21447754
90 1082.69555664 inf

epoch 1 : pseudo= 870.027893116, cost= inf, previous_cost= -, train_time = 1412.65254021 s
1
			 val_= 0.96177217357, test_time  = 0.567327976227 s

0 1008.90802002 1097.22888184
10 1148.34484863 inf
20 1188.77416992 inf
30 830.180358887 inf
40 989.983215332 1080.94189453
50 1165.68823242 inf
60 878.101196289 900.893859863
70 770.607116699 976.55090332
80 1060.52685547 1059.70800781
90 1047.16003418 inf

epoch 2 : pseudo= 1075.81285985, cost= inf, previous_cost= inf, train_time = 1405.69238281 s
1
			 val_= 0.947931417299, test_time  = 0.561805963516 s

0 1322.58581543 inf
10 1453.35913086 1897.55737305
20 889.750915527 990.330444336
30 890.891418457 842.42175293
40 554.725280762 613.603759766
50 673.

KeyboardInterrupt: 

In [ ]:
＃ Not used below

# Only Process once

In [1]:
''' 
[ Doing : ]
    - Read character from *.txt to build the corpus 

[ Result : ]

-- Without Develope Data:

Building corpus without develope data:
set([' ', "'", '-', '?', '_', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z'])
Total len is : 31
Done Write to [./dataCebuano/corpus.txt]

-- With Develope Data:

Building corpus with develope data:
set([' ', "'", ')', '(', '-', '?', '_', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z'])
Total len is : 33
Done Write to [./dataCebuano/corpus.txt]
'''

TRAINING_LABEL = "./dataCebuano/train.txt"
DEVELOPE_LABEL = "./dataCebuano/dev.txt"
WRITING_TO = "./dataCebuano/corpus.txt"

# Special part to build corpus
def buildCharacterCorpus(withDevelopeData = False) :
    corpus = set( )
    with open(TRAINING_LABEL, 'r') as ft:
        for lines in ft:
            for word in lines.split('\n')[0].split(' ')[1:]:
                if word[0] == '<':
                    # We take every <...> as <unk> 
                    unKnownTag = '?'
                    corpus.add( unKnownTag ) 
                else:
                    for char in word:
                        corpus.add(char.lower())
    
    if( withDevelopeData ) :
        with open(DEVELOPE_LABEL, 'r') as fd:
            for lines in fd:
                for word in lines.split('\n')[0].split(' ')[1:]:
                    if word[0] == '<':
                        unKnownTag = '?'
                        corpus.add( unKnownTag ) 
                        # corpus.add(word.split('\n')[0].lower())
                    else:
                        for char in word:
                            corpus.add(char.lower())
    
    corpus.add(' ')
    print "Building corpus with" + ("" if withDevelopeData else "out") + " develope data:"
    print corpus
    print "Total len is : " + str(len(corpus))
    
    
    with open(WRITING_TO, 'w+') as fw:
        for item in corpus:
            fw.write(item + '\n')        
    print "Done Write to [" + WRITING_TO + "]"
    return corpus

trainingCorpus = buildCharacterCorpus()



Building corpus without develope data:
set([' ', "'", '-', '?', '_', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z'])
Total len is : 31
Done Write to [./dataCebuano/corpus.txt]


In [10]:
# import theano
# from theano import tensor as T
# from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np

from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn import grid_search as gs
from sklearn.metrics import accuracy_score
from itertools import repeat

#proprocessor
from sklearn import preprocessing
from time import time

from itertools import repeat
import numpy as np
import random
from sklearn import preprocessing


def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def vectorized_result ( j , siz) :
    e = np.zeros((siz, 1))
    e[j] = 1.0
    return np.reshape( e, siz)

TRAINING_DATA   = './dataCebuano/tlkagk/train.dnn.fea'
# VALIDATION_DATA = './dataCebuano/dev.f0_ffv_fbank.fea'
TRAIN_ARK  = './dataCebuano/tlkagk/train.dnn.fea.lessThen500.ark'
# TRAIN_ARK  = './dataCebuano/train_Normalized_SELFVALID_LenLessThen600.ark'


def normalizeAndFilterData ( maxLength = 3000) :
    # Dataset
    x_data = []
    x_name = []
    record_seq = []

    with open( TRAINING_DATA, 'r') as f:
        for lines in f: 
            if '[' in lines :
                id = lines.split(' ')[0]
                x_name.append(id)
                seq = 0
            elif ']' in lines :
                if seq > maxLength :
                    x_name.pop()
                    del x_data[-seq:]
                else :
                    seq += 1
                    record_seq.append(seq)
                    x_data.append([float(x) for x in lines.split(' ') [2:-1] ])
            else :
                seq += 1
                x_data.append([float(x) for x in lines.split(' ') [2:-1] ])

    print "Normalized"
    x_data = preprocessing.scale( np.array(x_data) )

    # x_data[separate+1]
    # x_name
    bitch = record_seq.index(max(record_seq))
    print "Max len is ", record_seq[bitch]
    
    print "Each Length is ", len(x_data[bitch])
    print "Original Total training data", len(x_name)

    # trainF = open(TRAIN_ARK, 'w+')
    with open(TRAIN_ARK, 'w+') as trainF:
        currentSeq = 0
        for nameIndex in range( len(x_name) ):

            trainF.write(str(x_name[nameIndex])+"  [\n")

            for valueIndex in range( (record_seq[nameIndex]) ):
                for featureIndex in range( len(x_data[0]) ):
                    if ( featureIndex+1) != len(x_data[0] ) :
                        trainF.write( str(x_data[ currentSeq  + valueIndex][featureIndex]) + " ")
                    else:
                        trainF.write( str(x_data[ currentSeq  + valueIndex][featureIndex]) )
                if valueIndex+1 == (record_seq[nameIndex]):
                    trainF.write(" ]\n")
                else:
                    trainF.write("\n")
            currentSeq += record_seq[nameIndex]

    print "MIssion ComplEte"


normalizeAndFilterData(500)
# ORIGINAL 3918

Normalized
Max len is  501
Each Length is  90
Original Total training data 3918
MIssion ComplEte


array([[ 0.01      ,  0.002     ],
       [ 1.20000005,  1.10000002]], dtype=float32)